# Implementación Práctica y Exploración de RAG con Langchain


## 1. Implementacion Práctica de RAG

En esta seccion se implementa desde cero un sistema RAG usando Langchain


In [1]:
# Instalacion de liberias
# pip install langchain langchain_community langchain-text-splitters langchain-ollama langchain-chroma langchain-google-genai chromadb

import os

apikey = os.getenv("API_KEY")


### Carga y preparación de un documento de ejemplo

Para efectos de prueba, creamos un archivo textual con datos random.


In [2]:
documento_ejemplo = """
En un pueblo pequeño, donde las calles olían a pan recién horneado y las ventanas siempre estaban abiertas, vivía Tomás, un chico que coleccionaba sonidos. No objetos, no estampillas: sonidos.
Guardaba en frascos de vidrio el eco de una carcajada, el chasquido de una rama al partirse, el silbido del tren al amanecer. Cada frasco tenía una etiqueta y un color distinto, y cuando los agitaba, el sonido volvía a vibrar, como si nunca se hubiera ido.
Una tarde, mientras caminaba por la plaza, escuchó algo que jamás había oído: un silencio cálido, profundo, como un abrazo invisible. Era el silencio que aparece justo antes de que alguien diga algo importante.
Tomás corrió a su casa, tomó un frasco vacío y trató de capturarlo. Pero al abrirlo, el silencio se escapó entre sus dedos como si nada pudiera contenerlo.
Esa noche entendió que algunos sonidos son demasiado grandes para guardarlos. Algunos —como ese silencio especial— están destinados a llenarlo todo, incluso a uno mismo.
Y desde entonces, Tomás ya no coleccionó solo sonidos. También coleccionó momentos, porque descubrió que muchas veces son lo mismo.
"""

with open("El coleccionista de sonidos.txt", "w", encoding="utf-8") as f:
    f.write(documento_ejemplo)


### Pipeline RAG: Carga, Split, Embedding, Indexado y QA


In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI

loader = TextLoader("El coleccionista de sonidos.txt")
docs = loader.load()

splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=30)
chunks = splitter.split_documents(docs)

embedding_model = OllamaEmbeddings(model="nomic-embed-text")
db = Chroma.from_documents(chunks, embedding_model)
apik = apikey

try:
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key=apik)
except Exception as e:
    print(f"Google GenAI no disponible: {e}. Usa Ollama LLM si lo prefieres.")


c:\Users\IPF-2025\Desktop\proyectos\Python_para_ciencia_de_datos\Trag\RAG_langchain_ollama_tp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Función de pregunta-respuesta sobre el contenido usando RAG**


In [4]:
def rag_query(pregunta, db=db, llm=llm, k=3):
    docs = db.similarity_search(pregunta, k=k)
    contexto = "\n".join([d.page_content for d in docs])
    prompt = f"""Contexto:
{contexto}

Pregunta: {pregunta}
Respuesta:""".format(contexto=contexto, pregunta=pregunta)
    respuesta = llm.invoke(prompt)
    return respuesta.content if hasattr(respuesta, 'content') else respuesta

pregunta_ejemplo = "¿Que otra cosa colecciono tomas?"
print(rag_query(pregunta_ejemplo))


Tomás también coleccionó momentos.


---
## 2. uso de Modelos de Embedding y LLM

Se prueban distintos modelos de embeddings y LLM compatibles para observar como recupera documentos y como genera las respuestas.


In [5]:
from langchain_community.embeddings import SentenceTransformerEmbeddings

embedding_configs = [
    ("nomic-embed-text", OllamaEmbeddings(model="nomic-embed-text")),                  
    ("embeddinggemma", OllamaEmbeddings(model="embeddinggemma")),                              
]

query_text = "inteligencia artificial en agricultura"

for name, emb in embedding_configs:
    try:
        db_temp = Chroma.from_documents(chunks, emb, persist_directory=f"./chroma_{name}")
        docs_found = db_temp.similarity_search(query_text, k=2)
        print(f"\nEmbedding model: {name}\n-")
        for d in docs_found:
            print(f"> Fragmento: {d.page_content[:80]}...")
    except Exception as e:
        print(f"Error con embedding {name}: {e}")



Embedding model: nomic-embed-text
-
> Fragmento: En un pueblo pequeÃ±o, donde las calles olÃ­an a pan reciÃ©n horneado y las vent...

Embedding model: embeddinggemma
-
> Fragmento: En un pueblo pequeÃ±o, donde las calles olÃ­an a pan reciÃ©n horneado y las vent...


In [ ]:
import time
benchmark_results = []
query_text = "inteligencia artificial experta en cuentos"

for name, emb in embedding_configs:
    try:
        start_time = time.time()
        db_temp = Chroma.from_documents(chunks, emb, persist_directory=f"./chroma_{name}")
        docs_found = db_temp.similarity_search(query_text, k=2)
        elapsed = time.time() - start_time
        fragmentos = [d.page_content[:80] for d in docs_found]
        benchmark_results.append({
            "Modelo": name,
            "Tiempo (seg)": round(elapsed, 3),
            "Fragmento 1": fragmentos[0] if len(fragmentos) > 0 else "",
            "Fragmento 2": fragmentos[1] if len(fragmentos) > 1 else ""
        })
        print(f"\nEmbedding: {name} | Tiempo: {elapsed:.3f} seg")
        for i, frag in enumerate(fragmentos):
            print(f"Fragmento {i+1}: {frag}")
    except Exception as e:
        benchmark_results.append({"Modelo": name, "Tiempo (seg)": "Error", "Fragmento 1": str(e), "Fragmento 2": ""})
        print(f"Error con embedding {name}: {e}")

import pandas as pd
pd.DataFrame(benchmark_results)



Embedding: nomic-embed-text | Tiempo: 1.342 seg
Fragmento 1: En un pueblo pequeÃ±o, donde las calles olÃ­an a pan reciÃ©n horneado y las vent
Fragmento 2: En un pueblo pequeÃ±o, donde las calles olÃ­an a pan reciÃ©n horneado y las vent

Embedding: embeddinggemma | Tiempo: 1.835 seg
Fragmento 1: En un pueblo pequeÃ±o, donde las calles olÃ­an a pan reciÃ©n horneado y las vent
Fragmento 2: En un pueblo pequeÃ±o, donde las calles olÃ­an a pan reciÃ©n horneado y las vent


,Modelo,Tiempo (seg),Fragmento 1,Fragmento 2
0,nomic-embed-text,1.342,"En un pueblo pequeÃ±o, donde las calles olÃ­an...","En un pueblo pequeÃ±o, donde las calles olÃ­an..."
1,embeddinggemma,1.835,"En un pueblo pequeÃ±o, donde las calles olÃ­an...","En un pueblo pequeÃ±o, donde las calles olÃ­an..."


In [7]:
from langchain_ollama import ChatOllama

llm_models = [
    ("google-genai (gemini-2.5-flash)", llm),
    ("qwen2.5:0.5b", ChatOllama(model="qwen2.5:0.5b")),
]

for nombre, modelo_llm in llm_models:
    try:
        print(f"\n>>> {nombre}")
        respuesta = rag_query("¿De que trata el cuento?", db=db, llm=modelo_llm)
        print(respuesta)
    except Exception as e:
        print(f"Error usando modelo {nombre}: {e}")



>>> google-genai (gemini-2.5-flash)
El cuento trata sobre un niño llamado Tomás que colecciona sonidos en frascos. Un día descubre un silencio especial, un silencio "cálido y profundo", que intenta capturar sin éxito. A partir de esta experiencia, Tomás comprende que algunos sonidos son demasiado grandes para ser contenidos y que algunos momentos son tan valiosos como los sonidos mismos, por lo que decide coleccionar también momentos. En esencia, el cuento trata sobre el descubrimiento de la importancia de vivir el presente y apreciar las experiencias más allá de intentar poseerlas.

>>> qwen2.5:0.5b
El cuento trata sobre un niño chico llamado Tomás que es muy interesante en cuanto al arte. El protagonista, un joven de 12 años, busca coleccionar sus propios sonidos para luego guardarlos. Este se llama "sonido" y es parte del proceso de coleccionar cosas. Tomás vive en una casa pequeño en un pueblo lejano donde los chinos cocinan pan recién hurgado y las ventanas siempre están abiertas

---
## 3. Optimizacion del Separador de Texto

Se prueba variación en los parametros `chunk_size` y `chunk_overlap` del CharacterTextSplitter.


In [8]:
sizes = [(500, 100), (1000, 200)]

for cs, co in sizes:
    splitter = CharacterTextSplitter(chunk_size=cs, chunk_overlap=co)
    pedazos = splitter.split_documents(docs)
    print(f"chunk_size={cs}, chunk_overlap={co} => chunks: {len(pedazos)}")
    print(f"> Ejemplo de chunk: {pedazos[0].page_content[:60]}...\n")


chunk_size=500, chunk_overlap=100 => chunks: 1
> Ejemplo de chunk: En un pueblo pequeÃ±o, donde las calles olÃ­an a pan reciÃ©n...

chunk_size=1000, chunk_overlap=200 => chunks: 1
> Ejemplo de chunk: En un pueblo pequeÃ±o, donde las calles olÃ­an a pan reciÃ©n...



---
## 4. Gestionar Bases de Datos Vectoriales

Esta seccion es para la persistencia, colecciones y búsquedas avanzadas con Chroma.


In [9]:
persist_path = "./chroma_db"
db_persist = Chroma.from_documents(chunks, embedding_model, persist_directory=persist_path)

print(f"Base de datos Chroma guardada en {persist_path}")


Base de datos Chroma guardada en ./chroma_db


In [10]:
for d in chunks:
    d.metadata["origen"] = "El coleccionista de sonidos.txt"

db_filtrada = Chroma.from_documents(chunks, embedding_model)
results = db_filtrada.similarity_search_with_score("frascos", k=5, filter={"origen": "El coleccionista de sonidos.txt"})
for doc, score in results:
    print(f"Score: {score:.3f} | Fragmento: {doc.page_content[:70]}...")


Score: 1.078 | Fragmento: En un pueblo pequeÃ±o, donde las calles olÃ­an a pan reciÃ©n horneado ...


---
## 5. Refinamiento de Prompts

Experimentamos con diferentes plantillas y técnicas de prompt engineering para mejorar la calidad, relevancia y trazabilidad de las respuestas generadas por el LLM dentro del sistema RAG.


In [11]:
from langchain_core.prompts import PromptTemplate

base_prompt = """Eres un experto en Inteligencia Artificial especializado en análisis de textos.
Tu tarea es responder únicamente basándote en el contenido del bloque {context}.
Debes citar textualmente las partes del texto que fundamenten tu respuesta cuando sea pertinente.
No añadas información externa, no interpretes más allá de lo que el texto permite y no inventes datos ausentes.

Texto a analizar:
{context}

Pregunta sobre el texto:
{question}

Respuesta (basada estrictamente en el texto):"""
plantilla = PromptTemplate(template=base_prompt, input_variables=["context", "question"])

def rag_query_template(pregunta, db=db, llm=llm, k=3, prompt_template=plantilla):
    docs = db.similarity_search(pregunta, k=k)
    contexto = "\n".join([d.page_content for d in docs])
    prompt = prompt_template.format(context=contexto, question=pregunta)
    respuesta = llm.invoke(prompt)
    return respuesta.content if hasattr(respuesta, 'content') else respuesta

pregunta_2 = "¿Quien es tomás?"
print(rag_query_template(pregunta_2))


Tomás es "un chico que coleccionaba sonidos. No objetos, no estampillas: sonidos".
